**Economic Dispatch Seminar**

**Power Markets and Regulations**

**TA: Sahar Moghimian**

<!--NOTEBOOK_HEADER-->
*Most of the materials used for this notebook and our future seminars were developed by Prof. Michael Davidson and Prof. Jesse Jenkins and are used at the following courses:*

- *MAE / ENE 539 Optimization Methods for Energy Systems Engineering (Advanced Topics in Combustion I) [Princeton]*

- *MAE 243 Electric Power Systems Modeling [UC San Diego]*

*The full Github Repo with materials can be found* [here](https://github.com/Power-Systems-Optimization-Course/power-systems-optimization?tab=readme-ov-file#readme)

---
*This notebook has been adapted to Python/Pyomo and enhanced with real CAISO data, LMP extraction, and interactive exercises.*

# Economic Dispatch

_**[Power Systems Optimization](https://github.com/east-winds/power-systems-optimization)**_

_by Michael R. Davidson and Jesse D. Jenkins (last updated: October 4, 2022)_

This notebook will introduce a key operational model&mdash;economic dispatch (ED)&mdash;which minimizes the short-run production costs of meeting electricity demand from a given set of generators subject to various technical constraints.

We build up the model in several stages.

1. We begin with a single-time period and simple generator constraints just related to allowable output ranges. This allows us to compare the model with the intuition provided by the merit order supply curve (also known as the "dispatch stack").

2. We then add multiple time periods by considering the economic dispatch over an entire day. Due to the simple generator constraints, this simple model does not introduce any coupling across time.

3. More realism is added by considering engineering constraints that introduce time coupling&mdash;namely, ramp rates, which are an important limitation of many generators.

4. We examine the famous "duck curve" by adding more solar onto the system and explore the role that ramp rates play in the solution.

5. **NEW:** We extract **Locational Marginal Prices (LMPs)** from the demand constraint duals and compare with real CAISO market prices.

6. **NEW:** We compare our model results with **actual CAISO generation and demand data** from 2023-2024.

7. **NEW:** We "break the model" in several instructive ways and build a **comparison table** across all scenarios.

## Introduction to economic dispatch

Every day, system operators need to decide how to meet demand from a large variety of generators with different costs and engineering requirements. Prior to large-scale computational capabilities, this was accomplished by "priority lists" or similar heuristics to determine the ordering of plants. However, with modern optimization algorithms, operators can find the global minimum of production costs&mdash;and even incorporate coupling of engineering processes across time periods.

**Economic dispatch** (ED) is the problem of minimizing short-run costs of production in order to meet a given demand, considering relevant engineering constraints of the generators. It *does not* fully capture the physics of electricity flows, which will further constrain the feasible space and which we will come back to in later notebooks on optimal power flow, nor does it account for decisions and constraints related to turning on or "committing" large thermal generators, which we'll discuss in our next notebook on unit commitment. Typically, *it does* incorporate some type of network representation, which we have ignored here by considering only a single bus. More on the networks in the notebook on optimal power flow.

## Single-time period, simple generator constraints

We will first examine the case where we are optimizing dispatch for a single snapshot in time, with only very simple constraints on the generators.
    
$$
\begin{align}
\min \ & \sum_{g \in G} VarCost_g \times GEN_g & \\
\text{s.t.} & \\
 & \sum_{g} GEN_g = Demand & \\
 & GEN_g \leq Pmax_g & \forall \quad g \in G \\
 & GEN_g \geq Pmin_g & \forall \quad g \in G \\
\end{align}
$$

The **decision variable** in the above problem is:

- $GEN_{g}$, the generation (in MW) produced by each generator, $g$

The **parameters** are:

- $Pmin_g$, the minimum operating bounds for the generator (based on engineering or natural resource constraints)
- $Pmax_g$, the maximum operating bounds for the generator (based on engineering or natural resource constraints)
- $Demand$, the demand (in MW)

(For this simple problem, we will let $Pmin_g=0$ and revisit this later in the unit commitment notebook.)

In addition, we have:

$$
VarCost_g = VarOM_g + HeatRate_g \times FuelCost_g
$$

- **VarOM_g**: This is the variable operation and maintenance cost per unit of energy produced (typically per megawatt-hour, or MWh). It covers costs that vary with production levels, such as maintenance that scales with wear and tear from generation.

- **HeatRate$_g$**: This represents the efficiency of the generator and is given in MMBtu/MWh (million British thermal units per megawatt-hour). It tells us how much fuel energy is needed to produce one MWh of electricity. A lower heat rate means higher efficiency.

- **FuelCost$_g$**: This is the cost of fuel per MMBtu for the generator.

Note that, in contrast with the basic capacity expansion problem (long term planning), we are not concerned with fixed costs.

Why not?

The simplest response is that these costs have already been incurred and regardless of how much a generator produces, its fixed costs will not change. These costs are thus "[sunk costs](https://en.wikipedia.org/wiki/Sunk_cost)" and are constant in the objective function. Our optimal decision variables would not change by adjusting this constant up or down. Therefore, we can safely ignore fixed costs for the purposes of finding optimal dispatch. (We will still have to consider them to calculate producer profits, however.)

> [A famous electricity regulator](https://en.wikipedia.org/wiki/Ignacio_J._P%C3%A9rez_Arriaga), noting that it is frequently misunderstood and ignored even by experienced market participants, has called the application of the "[sunk cost fallacy](https://en.wikipedia.org/wiki/Sunk_cost#Fallacy_effect)" to electricity systems "[Grandma's Inheritance Theorem](http://link.springer.com/content/pdf/bbm%3A978-1-4471-5034-3%2F1.pdf)." In this hypothetical scenario, if you were to inherit a diamond ring or perhaps a financial contract-for-differences that pays out relative to cleared market prices (you have a very sophisticated grandma in this scenario!), you should not be tempted to change your production strategy.

Now, let's implement ED.

### 1. Install and load packages

In [ ]:
# Install required packages (run this cell once)
!pip install -q pyomo highspy pandas matplotlib plotly gridstatus

In [ ]:
import pyomo.environ as pyo
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', 120)
pd.set_option('display.width', 120)

### 2. Load and format data

We will use data based on San Diego Gas and Electric (SDG&E, via the [PowerGenome](https://github.com/gschivley/PowerGenome) data platform) plus a few neighboring generators, consisting of:

- 25 generators (including some clustering of smaller generators, and excluding behind-the-meter solar)
- estimated hourly demand for 2020 (net load at the transmission substation level after subtracting 600MW of behind-the-meter solar from original demand)
- variable generation capacity factors
- estimated natural gas fuel costs

In [ ]:
base_url = "https://raw.githubusercontent.com/Power-Systems-Optimization-Course/power-systems-optimization/master/Notebooks/ed_data/"

gen_info = pd.read_csv(base_url + "Generators_data.csv")
fuels = pd.read_csv(base_url + "Fuels_data.csv")
loads = pd.read_csv(base_url + "Demand.csv")
gen_variable = pd.read_csv(base_url + "Generators_variability.csv")

# Rename all columns to lowercase (by convention)
for df in [gen_info, fuels, loads, gen_variable]:
    df.columns = df.columns.str.lower()

**Construct generator dataframe**

In [ ]:
# Keep only the columns relevant to our ED model
gen_info = gen_info.iloc[:, list(range(26)) + [gen_info.columns.get_loc('stor')]]
gen_df = gen_info.merge(fuels, on='fuel', how='outer')  # load in fuel costs
gen_df.rename(columns={'cost_per_mmbtu': 'fuel_cost'}, inplace=True)
gen_df['fuel_cost'] = gen_df['fuel_cost'].fillna(0)

# Create "is_variable" column to indicate if this is a variable generation source
variable_resources = ['onshore_wind_turbine', 'small_hydroelectric', 'solar_photovoltaic']
gen_df['is_variable'] = gen_df['resource'].isin(variable_resources)

# Create full name of generator (including geographic location and cluster number)
gen_df['gen_full'] = (gen_df['region'] + '_' + gen_df['resource'] + '_' + gen_df['cluster'].astype(str) + '.0').str.lower()

# Remove generators with no capacity (new build options for capacity expansion)
gen_df = gen_df[gen_df['existing_cap_mw'] > 0].reset_index(drop=True)

### Long and wide data format example

In [ ]:
gen_df.iloc[:5, :5]

In [ ]:
# Convert from wide to long format (equivalent to Julia's stack)
df = gen_df.iloc[:5, :5].copy()
df_long = df.melt(
    id_vars=['r_id', 'resource', 'region'],
    value_vars=['existing_cap_mw', 'num_units'],
    var_name='var',
    value_name='val'
)
df_long

**Modify load and variable generation dataframes**

1. Convert from GMT to GMT-8

The load and variable generation files are in GMT. We want to convert to local California time. Here, we will ignore daylight savings and simply subtract 8 hours.

To accomplish this, we 'wrap' the last 8 hours of the data series around to the front of the series.

In [ ]:
gen_variable['hour'] = (gen_variable['hour'] - 9) % 8760 + 1
gen_variable = gen_variable.sort_values('hour').reset_index(drop=True)

loads['hour'] = (loads['hour'] - 9) % 8760 + 1
loads = loads.sort_values('hour').reset_index(drop=True)

2. Convert from "wide" to "long" format:

The file is in "wide" format, which has separate columns for values from different types of generators. This has more human readability, but often we want to have it in "long" format for computation.

"Long" format refers to a dataframe with a separate row entry for every value. In Python, we use the pandas [`melt`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.melt.html) function to accomplish this.

In [ ]:
gen_variable_long = gen_variable.melt(
    id_vars='hour',
    var_name='gen_full',
    value_name='cf'
)

# Let's look at the first 6 entries of a wind resource for example
gen_variable_long[gen_variable_long['gen_full'] == 'wec_sdge_onshore_wind_turbine_1.0'].head(6)

**Extract single time from the data**
(we will get to the full set later)

- demand
- variable generation

Note: variable generation profiles are often encoded in terms of hourly maximum capacity factor, expressed as a per unit value or percentage of installed capacity. Hourly capacity factor is then later multiplied by installed capacity to yield a maximum hourly generation for each resource.

In [ ]:
hr = 2416  # pick 4pm on a spring day
loads_single = loads[loads['hour'] == hr].drop(columns='hour')
var_cf_single = gen_variable_long[gen_variable_long['hour'] == hr].drop(columns='hour')
var_cf_single

---
### Quiz 1: Before we solve

**Think about the following before running the next cells:**

1. Looking at the generator data, which generators do you expect to be dispatched first? Why?
2. If demand is ~2,500 MW, do we need *all* generators to meet it?
3. What is the variable cost of solar and wind generation? What does this imply for their position in the merit order?

*Write down your answers, then continue to see if the model confirms your intuition.*

---

### 3. Create solver function

Here, we introduce a functionalized version of the optimization problem. We pass the problem parameters to the function, and it constructs and solves the model, returning the result. This will be useful when we wish to call the same basic model multiple times, e.g. for conducting sensitivities on parameters.

We use the **HiGHS** solver (high performance software for linear optimization): for large-scale sparse linear programming (LP), mixed-integer programming (MIP), and quadratic programming (QP) models.

In [ ]:
def economic_dispatch_single(gen_df, loads, gen_variable):
    """
    Solve economic dispatch problem (single-time period, single-zone).
    
    Parameters:
        gen_df       -- DataFrame with generator info
        loads        -- DataFrame with load info (single row)
        gen_variable -- capacity factors of variable generators (in 'long' format)
    """
    model = pyo.ConcreteModel()
    model.dual = pyo.Suffix(direction=pyo.Suffix.IMPORT)

    # Sets
    G_var = list(gen_df[gen_df['is_variable'] == True]['r_id'])
    G_nonvar = list(gen_df[gen_df['is_variable'] == False]['r_id'])
    G = list(gen_df['r_id'])
    model.G = pyo.Set(initialize=G)

    # Generator capacity factor for variable generators
    gen_var_cf = gen_variable.merge(
        gen_df[gen_df['is_variable'] == True][['r_id', 'gen_full', 'existing_cap_mw']],
        on='gen_full', how='inner'
    )

    gen_lookup = gen_df.set_index('r_id')

    # Decision variables
    model.GEN = pyo.Var(model.G, within=pyo.NonNegativeReals)

    # Objective function
    def obj_rule(m):
        return (
            sum(
                (gen_lookup.loc[i, 'heat_rate_mmbtu_per_mwh'] * gen_lookup.loc[i, 'fuel_cost'] +
                 gen_lookup.loc[i, 'var_om_cost_per_mwh']) * m.GEN[i]
                for i in G_nonvar
            ) +
            sum(
                gen_lookup.loc[i, 'var_om_cost_per_mwh'] * m.GEN[i]
                for i in G_var
            )
        )
    model.objective = pyo.Objective(rule=obj_rule, sense=pyo.minimize)

    # Demand constraint
    demand_val = loads.iloc[0]['demand']
    model.cDemand = pyo.Constraint(
        expr=sum(model.GEN[i] for i in G) == demand_val
    )

    # Capacity constraint (non-variable generation)
    model.cap_nonvar = pyo.ConstraintList()
    for i in G_nonvar:
        model.cap_nonvar.add(model.GEN[i] <= gen_lookup.loc[i, 'existing_cap_mw'])

    # Variable generation capacity constraint
    model.cap_var = pyo.ConstraintList()
    for _, row in gen_var_cf.iterrows():
        model.cap_var.add(model.GEN[row['r_id']] <= row['cf'] * row['existing_cap_mw'])

    # Solve
    solver = pyo.SolverFactory('appsi_highs')
    result = solver.solve(model, tee=True)

    # Check feasibility
    status = str(result.solver.termination_condition)
    if status != 'optimal':
        print(f"\n*** WARNING: Solver status = {status} ***")
        return {'solution': pd.DataFrame(), 'cost': None, 'price': None, 'status': status}

    # Extract solution
    solution = pd.DataFrame({
        'r_id': gen_df['r_id'].values,
        'resource': gen_df['resource'].values,
        'gen': [pyo.value(model.GEN[i]) for i in gen_df['r_id']]
    })

    # Extract LMP (dual of demand constraint = market clearing price)
    try:
        lmp = model.dual[model.cDemand]
    except KeyError:
        lmp = None

    return {
        'solution': solution,
        'cost': pyo.value(model.objective),
        'price': lmp,
        'status': status,
    }

### 4. Solve and print data

In [ ]:
solution = economic_dispatch_single(gen_df, loads_single, var_cf_single)

In [ ]:
solution['solution']

In [ ]:
sol = solution['solution']
print(f"Total cost: ${solution['cost']:,.2f}")
print(f"Market clearing price (LMP): ${solution['price']:.2f}/MWh")
print(f"\nMax generation: {sol.loc[sol['gen'].idxmax(), 'resource']} = {sol['gen'].max():.1f} MW")
print(f"Generators at zero: {(sol['gen'] == 0).sum()} out of {len(sol)}")

### Understanding the LMP (Locational Marginal Price)

The **dual value** of the demand constraint tells us the **marginal cost of serving one additional MW of demand**. In a single-bus model like ours, this is the system-wide electricity price, equivalent to the variable cost of the *marginal generator*.

This is exactly how wholesale electricity markets work: all generators are paid the **market clearing price** (the price set by the most expensive generator that is dispatched), not their individual costs.

We will explore LMPs more thoroughly in later sections. For now, note that the LMP equals the variable cost of the marginal generator on the supply curve.

We will now plot the **supply curve** (also known as the **dispatch stack**) for this single period to illustrate the concept of **merit order**.

In [ ]:
supply_curve = gen_df.merge(var_cf_single, on='gen_full', how='left')
supply_curve['varcost'] = 0.0
supply_curve['cap'] = 0.0

# Variable cost and capacity for non-variable generators
nonvar_mask = supply_curve['is_variable'] == False
supply_curve.loc[nonvar_mask, 'varcost'] = (
    supply_curve.loc[nonvar_mask, 'heat_rate_mmbtu_per_mwh'] *
    supply_curve.loc[nonvar_mask, 'fuel_cost'] +
    supply_curve.loc[nonvar_mask, 'var_om_cost_per_mwh']
)
supply_curve.loc[nonvar_mask, 'cap'] = supply_curve.loc[nonvar_mask, 'existing_cap_mw']

# Variable cost and capacity for variable generators
var_mask = supply_curve['is_variable'] == True
supply_curve.loc[var_mask, 'varcost'] = supply_curve.loc[var_mask, 'var_om_cost_per_mwh']
supply_curve.loc[var_mask, 'cap'] = (
    supply_curve.loc[var_mask, 'existing_cap_mw'] * supply_curve.loc[var_mask, 'cf']
)

supply_curve = supply_curve.sort_values('varcost').reset_index(drop=True)

Now, let's plot this as a supply curve where:

- width = capacity of the generator (i.e., $Pmax$)
- height = variable cost of the generator ($VarCost$)

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))

demand_val = loads_single.iloc[0]['demand']
marg_gen = 0
marg_price = 0
x = 0

colors = plt.cm.tab20(np.linspace(0, 1, len(supply_curve)))

for idx in range(len(supply_curve)):
    width = supply_curve.iloc[idx]['cap']
    height = supply_curve.iloc[idx]['varcost']
    rect = mpatches.FancyBboxPatch((x, 0), width, height, alpha=0.5,
                                    boxstyle="square,pad=0", facecolor=colors[idx])
    ax.add_patch(rect)
    if x < demand_val and x + width > demand_val:
        marg_gen = idx
        marg_price = height
    x += width

ax.axvline(x=demand_val, color='black', linewidth=3, label='Demand')
ax.axhline(y=marg_price, color='blue', linewidth=3, label=f'SRMC = ${marg_price:.2f}/MWh')
ax.axhline(y=solution['price'], color='red', linewidth=2, linestyle='--', label=f'LMP (dual) = ${solution["price"]:.2f}/MWh')
ax.set_xlim(0, x)
ax.set_ylim(0, supply_curve['varcost'].max() * 1.1)
ax.set_xlabel('Capacity (MW)')
ax.set_ylabel('Marginal cost ($/MWh)')
ax.set_title('Supply and Demand Curves with LMP')
ax.legend(loc='upper right')
plt.tight_layout()
plt.show()

In the above, everything to the left of demand is dispatched at full capacity, except for the last generator (the "marginal generator") where demand stops. Everything to the right is not dispatched (=0). Note that the **LMP (red dashed) matches the SRMC (blue)** of the marginal generator.

In [ ]:
print("Marginal generator:")
supply_curve.iloc[marg_gen][['r_id', 'resource', 'varcost', 'existing_cap_mw']]

In [ ]:
marg_rid = supply_curve.iloc[marg_gen]['r_id']
marg_dispatch = solution['solution'][solution['solution']['r_id'] == marg_rid]['gen'].values[0]
print(f"Marginal generator dispatch: {marg_dispatch:.1f} MW (out of {supply_curve.iloc[marg_gen]['cap']:.1f} MW available)")

**Combine by generation type**

In [ ]:
sol_gen = solution['solution'].groupby('resource', as_index=False)['gen'].sum()
sol_gen.rename(columns={'gen': 'gen_sum'}, inplace=True)

# Add back BTM solar
btm_cf = var_cf_single[var_cf_single['gen_full'] == 'wec_sdge_solar_photovoltaic_1.0']['cf'].values[0]
btm = pd.DataFrame({'resource': ['solar_photovoltaic_btm'], 'gen_sum': [btm_cf * 600]})
sol_gen_btm = pd.concat([sol_gen, btm], ignore_index=True)
sol_gen_btm.round(3)

Hence, the majority of energy is being met by combined cycle natural gas turbines (CCGT), second by solar PV, and third by behind-the-meter solar.

---
### Quiz 2: Interpreting the LMP

**Think about these questions:**

1. The LMP (market clearing price) is set by the marginal generator. If demand *increases* by 100 MW, which generator would supply the extra power? Would the LMP change?
2. Solar and wind generators have zero (or near-zero) variable cost. Under this market design, at what price are they paid? Is this above or below their variable cost? What does this mean for their profits?
3. If we *doubled* the solar capacity, what would happen to the LMP at this hour? (Think about which generator becomes marginal.)

*We will verify answer #3 later in the duck curve section.*

---

## Multiple-time period, simple generator constraints

We neglected the time dimension above and now we will add it back in. The key changes to the model arise from adding an additional index for time:

$$
\begin{align}
\min \ & \sum_{g \in G, t \in T} VarCost_g \times GEN_{g,t} & \\
\text{s.t.} & \\
 & \sum_{g} GEN_{g,t} = Demand_t & \forall \quad t \in T \\
 & GEN_{g,t} \leq Pmax_{g,t} & \forall \quad g \in G , t \in T \\
 & GEN_{g,t} \geq Pmin_{g,t} & \forall \quad g \in G , t \in T
\end{align}
$$


Note that for conventional generators, we often have $Pmax_{g,t} = Pmax_{g}$ and $Pmin_{g,t} = Pmin_{g}$ constant over time. (We will consider unit commitment later, which will alter this logic.)

For variable renewable generators, it is natural that $Pmax_{g,t}$ varies with time (i.e., based on solar irradiation or wind speeds). $Pmin_{g,t}$ for wind and solar resources is usually 0, but for hydropower resources, minimum streamflow constraints can produce a time-variant parameter.

---
### Quiz 3: Before adding time

**Predict:**

1. At what hour of the day do you expect demand to be highest? Lowest?
2. At what hour do you expect the electricity price (LMP) to be highest? Is it the same hour as peak demand?
3. Without ramp constraints, can we solve each hour independently? Why or why not?

---

### 3. Create solver function

(We reuse steps 1 and 2 above to load packages and data.)

In [ ]:
def economic_dispatch_multi(gen_df, loads, gen_variable):
    """
    Solve economic dispatch problem (multi-time period, single-zone).
    Returns solution, cost, hourly LMPs, and solver status.
    """
    model = pyo.ConcreteModel()
    model.dual = pyo.Suffix(direction=pyo.Suffix.IMPORT)

    # Define sets
    G_var = list(gen_df[gen_df['is_variable'] == True]['r_id'])
    G_nonvar = list(gen_df[gen_df['is_variable'] == False]['r_id'])
    G = list(gen_df['r_id'])
    T = list(loads['hour'])

    model.G = pyo.Set(initialize=G)
    model.T = pyo.Set(initialize=T)

    gen_var_cf = gen_variable.merge(
        gen_df[gen_df['is_variable'] == True][['r_id', 'gen_full', 'existing_cap_mw']],
        on='gen_full', how='inner'
    )

    gen_lookup = gen_df.set_index('r_id')
    loads_lookup = loads.set_index('hour')

    # Decision variables
    model.GEN = pyo.Var(model.G, model.T, within=pyo.NonNegativeReals)

    # Objective function
    def obj_rule(m):
        return (
            sum(
                (gen_lookup.loc[i, 'heat_rate_mmbtu_per_mwh'] * gen_lookup.loc[i, 'fuel_cost'] +
                 gen_lookup.loc[i, 'var_om_cost_per_mwh']) * m.GEN[i, t]
                for i in G_nonvar for t in T
            ) +
            sum(
                gen_lookup.loc[i, 'var_om_cost_per_mwh'] * m.GEN[i, t]
                for i in G_var for t in T
            )
        )
    model.objective = pyo.Objective(rule=obj_rule, sense=pyo.minimize)

    # Demand constraint
    model.cDemand = pyo.Constraint(model.T)
    for t in T:
        model.cDemand[t] = sum(model.GEN[i, t] for i in G) == loads_lookup.loc[t, 'demand']

    # Capacity constraints (non-variable generation)
    model.cap_nonvar = pyo.ConstraintList()
    for i in G_nonvar:
        for t in T:
            model.cap_nonvar.add(model.GEN[i, t] <= gen_lookup.loc[i, 'existing_cap_mw'])

    # Variable generation capacity constraints
    model.cap_var = pyo.ConstraintList()
    for _, row in gen_var_cf.iterrows():
        model.cap_var.add(
            model.GEN[row['r_id'], row['hour']] <= row['cf'] * row['existing_cap_mw']
        )

    # Solve
    solver = pyo.SolverFactory('appsi_highs')
    result = solver.solve(model, tee=True)

    status = str(result.solver.termination_condition)
    if status != 'optimal':
        print(f"\n*** WARNING: Solver status = {status} ***")
        return {'solution': pd.DataFrame(), 'cost': None, 'lmps': pd.DataFrame(), 'status': status}

    # Extract solution
    records = []
    for i in G:
        for t in T:
            records.append({'r_id': i, 'hour': t, 'gen': pyo.value(model.GEN[i, t])})
    solution_df = pd.DataFrame(records)

    # Extract hourly LMPs
    lmps = pd.DataFrame({
        'hour': T,
        'lmp': [model.dual[model.cDemand[t]] for t in T]
    })

    return {
        'solution': solution_df,
        'cost': pyo.value(model.objective),
        'lmps': lmps,
        'status': status,
    }

### 4. Solve a day's economic dispatch

We will subset our year-long dataset to a single spring day and solve.

In [ ]:
n = 100
T_period = list(range(n * 24 + 1, (n + 1) * 24 + 1))

loads_multi = loads[loads['hour'].isin(T_period)].copy()
gen_variable_multi = gen_variable_long[gen_variable_long['hour'].isin(T_period)].copy()

gen_df_sens = gen_df.copy()

solution_multi = economic_dispatch_multi(gen_df_sens, loads_multi, gen_variable_multi)

In [ ]:
# Process and plot results with BTM solar
def process_and_plot(solution, gen_df, gen_variable_multi, T_period, title=''):
    """Helper to add BTM solar and create stacked area plot."""
    sol_gen = solution['solution'].merge(gen_df[['r_id', 'resource']], on='r_id')
    sol_gen = sol_gen.groupby(['resource', 'hour'], as_index=False)['gen'].sum()
    sol_gen.rename(columns={'gen': 'gen_sum'}, inplace=True)

    sol_gen_btm = sol_gen.copy()
    sol_gen_btm.loc[sol_gen_btm['resource'] == 'solar_photovoltaic', 'resource'] = '_solar_photovoltaic'
    sol_gen_btm.loc[sol_gen_btm['resource'] == 'onshore_wind_turbine', 'resource'] = '_onshore_wind_turbine'
    sol_gen_btm.loc[sol_gen_btm['resource'] == 'small_hydroelectric', 'resource'] = '_small_hydroelectric'

    btm_cf = gen_variable_multi[gen_variable_multi['gen_full'] == 'wec_sdge_solar_photovoltaic_1.0'][['hour', 'cf']].copy()
    btm = pd.DataFrame({
        'resource': '_solar_photovoltaic_btm',
        'hour': btm_cf['hour'].values,
        'gen_sum': btm_cf['cf'].values * 600
    })
    sol_gen_btm = pd.concat([sol_gen_btm, btm], ignore_index=True)
    sol_gen_btm = sol_gen_btm.sort_values(['hour', 'resource']).reset_index(drop=True)

    fig = px.area(sol_gen_btm, x='hour', y='gen_sum', color='resource',
                  title=title or 'Stacked Generation by Resource',
                  width=1000, height=500)
    fig.show()
    return sol_gen_btm

sol_gen_btm_base = process_and_plot(solution_multi, gen_df, gen_variable_multi, T_period,
                                     'Stacked Generation (No Ramp Constraints)')

### Hourly LMPs (Electricity Prices)

Now let's plot the hourly electricity prices alongside demand. The LMP at each hour is the dual of the demand balance constraint.

In [ ]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    go.Scatter(x=loads_multi['hour'], y=loads_multi['demand'],
               name='Demand (MW)', line=dict(color='black', width=2)),
    secondary_y=False
)

fig.add_trace(
    go.Scatter(x=solution_multi['lmps']['hour'], y=solution_multi['lmps']['lmp'],
               name='LMP ($/MWh)', line=dict(color='red', width=2)),
    secondary_y=True
)

fig.update_layout(title='Demand and LMP Over a Spring Day (No Ramp Constraints)',
                  width=1000, height=500)
fig.update_xaxes(title_text='Hour')
fig.update_yaxes(title_text='Demand (MW)', secondary_y=False)
fig.update_yaxes(title_text='LMP ($/MWh)', secondary_y=True)
fig.show()

Notice how the LMP changes throughout the day as different generators become marginal. During midday when solar is abundant, cheaper generators are marginal. During evening peak demand, more expensive generators must be dispatched.

## Multiple-time period, complex generator constraints with time coupling

The above problem did not include more than one time index per constraint. Hence, we could actually solve this each time period separately without regard to what is happening before or after. (These are known as "separable" problems.)

We now introduce time coupling into the model, by considering ramp rates, which limit how much generators can change output from one period to the next (an engineering constraint).

The new model:

$$
\begin{align}
\min \ & \sum_{g \in G, t \in T} VarCost_g \times GEN_{g,t} & \\
\text{s.t.} & \\
 & \sum_{g} GEN_{g,t} = Demand_t & \forall \quad t \in T \\
 & GEN_{g,t} \leq Pmax_{g,t} & \forall \quad g \in G , t \in T \\
 & GEN_{g,t} \geq Pmin_{g,t} & \forall \quad g \in G , t \in T \\
 & GEN_{g,t+1} - GEN_{g,t} \leq RampUp_{g} & \forall \quad g \in G , t = 1..T-1 \\
  & GEN_{g,t} - GEN_{g,t+1} \leq RampDn_{g} & \forall \quad g \in G , t = 1..T-1
\end{align}
$$

---
### Quiz 4: Before adding ramp constraints

**Predict:**

1. Will the total daily cost increase or decrease after adding ramp constraints? Why?
2. Ramp constraints limit how fast generators can change output. During which hours of the day do you think ramp constraints will be most binding?
3. If a cheap combined-cycle plant can't ramp up fast enough to meet the evening peak, what happens?

---

In [ ]:
def economic_dispatch_multi_time(gen_df, loads, gen_variable):
    """
    Solve economic dispatch problem with ramp constraints
    (multi-time period, single-zone). Returns solution, cost, LMPs, and ramp marginals.
    """
    model = pyo.ConcreteModel()
    model.dual = pyo.Suffix(direction=pyo.Suffix.IMPORT)

    # Define sets
    G_var = list(gen_df[gen_df['is_variable'] == True]['r_id'])
    G_nonvar = list(gen_df[gen_df['is_variable'] == False]['r_id'])
    G = list(gen_df['r_id'])
    T = list(loads['hour'])
    T_red = T[:-1]

    model.G = pyo.Set(initialize=G)
    model.T = pyo.Set(initialize=T, ordered=True)

    gen_var_cf = gen_variable.merge(
        gen_df[gen_df['is_variable'] == True][['r_id', 'gen_full', 'existing_cap_mw']],
        on='gen_full', how='inner'
    )

    gen_lookup = gen_df.set_index('r_id')
    loads_lookup = loads.set_index('hour')

    # Decision variables
    model.GEN = pyo.Var(model.G, model.T, within=pyo.NonNegativeReals)

    # Objective function
    def obj_rule(m):
        return (
            sum(
                (gen_lookup.loc[i, 'heat_rate_mmbtu_per_mwh'] * gen_lookup.loc[i, 'fuel_cost'] +
                 gen_lookup.loc[i, 'var_om_cost_per_mwh']) * m.GEN[i, t]
                for i in G_nonvar for t in T
            ) +
            sum(
                gen_lookup.loc[i, 'var_om_cost_per_mwh'] * m.GEN[i, t]
                for i in G_var for t in T
            )
        )
    model.objective = pyo.Objective(rule=obj_rule, sense=pyo.minimize)

    # Demand constraint
    model.cDemand = pyo.Constraint(model.T)
    for t in T:
        model.cDemand[t] = sum(model.GEN[i, t] for i in G) == loads_lookup.loc[t, 'demand']

    # Capacity constraints (non-variable generation)
    model.cap_nonvar = pyo.ConstraintList()
    for i in G_nonvar:
        for t in T:
            model.cap_nonvar.add(model.GEN[i, t] <= gen_lookup.loc[i, 'existing_cap_mw'])

    # Variable generation capacity constraints
    model.cap_var = pyo.ConstraintList()
    for _, row in gen_var_cf.iterrows():
        model.cap_var.add(
            model.GEN[row['r_id'], row['hour']] <= row['cf'] * row['existing_cap_mw']
        )

    # Ramp up constraints
    model.ramp_up = pyo.Constraint(model.G, T_red)
    for i in G:
        for idx, t in enumerate(T_red):
            t_next = T[idx + 1]
            model.ramp_up[i, t] = (
                model.GEN[i, t_next] - model.GEN[i, t] <=
                gen_lookup.loc[i, 'existing_cap_mw'] * gen_lookup.loc[i, 'ramp_up_percentage']
            )

    # Ramp down constraints
    model.ramp_dn = pyo.Constraint(model.G, T_red)
    for i in G:
        for idx, t in enumerate(T_red):
            t_next = T[idx + 1]
            model.ramp_dn[i, t] = (
                model.GEN[i, t] - model.GEN[i, t_next] <=
                gen_lookup.loc[i, 'existing_cap_mw'] * gen_lookup.loc[i, 'ramp_dn_percentage']
            )

    # Solve
    solver = pyo.SolverFactory('appsi_highs')
    result = solver.solve(model, tee=True)

    status = str(result.solver.termination_condition)
    if status != 'optimal':
        print(f"\n*** WARNING: Solver status = {status} ***")
        return {'solution': pd.DataFrame(), 'cost': None, 'lmps': pd.DataFrame(),
                'marginals': pd.DataFrame(), 'status': status}

    # Extract solution
    records = []
    for i in G:
        for t in T:
            records.append({'r_id': i, 'hour': t, 'gen': pyo.value(model.GEN[i, t])})
    solution_df = pd.DataFrame(records)

    # Extract hourly LMPs
    lmps = pd.DataFrame({
        'hour': T,
        'lmp': [model.dual[model.cDemand[t]] for t in T]
    })

    # Extract ramp up constraint duals
    ramp_marginals = []
    for i in G:
        for t in T_red:
            try:
                dual_val = model.dual[model.ramp_up[i, t]]
            except (KeyError, AttributeError):
                dual_val = 0.0
            ramp_marginals.append({'r_id': i, 'hour': t, 'dual': dual_val})
    marginals_df = pd.DataFrame(ramp_marginals)

    return {
        'solution': solution_df,
        'cost': pyo.value(model.objective),
        'lmps': lmps,
        'marginals': marginals_df,
        'status': status,
    }

### 4. Solve a day's economic dispatch (with ramp constraints)

In [ ]:
solution_ramp = economic_dispatch_multi_time(gen_df, loads_multi, gen_variable_multi)

In [ ]:
sol_gen_btm_ramp = process_and_plot(solution_ramp, gen_df, gen_variable_multi, T_period,
                                     'Stacked Generation (With Ramp Constraints)')

### Compare LMPs: with vs. without ramp constraints

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=solution_multi['lmps']['hour'], y=solution_multi['lmps']['lmp'],
                         name='Without ramps', line=dict(dash='dash')))
fig.add_trace(go.Scatter(x=solution_ramp['lmps']['hour'], y=solution_ramp['lmps']['lmp'],
                         name='With ramps'))
fig.update_layout(title='LMP Comparison: With vs. Without Ramp Constraints',
                  xaxis_title='Hour', yaxis_title='LMP ($/MWh)',
                  width=1000, height=500)
fig.show()

print(f"Cost without ramps: ${solution_multi['cost']:,.2f}")
print(f"Cost with ramps:    ${solution_ramp['cost']:,.2f}")
print(f"Cost increase:      ${solution_ramp['cost'] - solution_multi['cost']:,.2f} "
      f"({(solution_ramp['cost'] - solution_multi['cost'])/solution_multi['cost']*100:.2f}%)")

### 5. Run sensitivity on solar capacity &mdash; The Duck Curve

Let's now examine what happens if we increase utility solar capacity from the current 500MW up to 3,500MW.

The large reduction in net load during the day and the loss of solar in the late afternoon timed with the rise in the evening peak is sometimes called the **"duck curve"** because of their passing resemblance to a duck's belly and head:

<img src="https://www.energy.gov/sites/prod/files/styles/borealis_photo_gallery_large_respondmedium/public/CAISO_DuckCurve_720_469_80.jpg?itok=99uYAxGo" style="width: 450px; height: auto" align="left">

---
### Quiz 5: Before the Duck Curve

**Predict:**

1. If we increase solar from 500 MW to 3,500 MW, what happens to midday LMPs? What about evening LMPs?
2. Will the total daily cost go up or down?
3. With 3,500 MW of solar, is it possible that some solar generation gets "wasted" (i.e., available solar exceeds demand)? What would happen to the LMP in that case?

---

In [ ]:
# Increase solar photovoltaic capacity
gen_df_duck = gen_df.copy()
gen_df_duck.loc[gen_df_duck['resource'] == 'solar_photovoltaic', 'existing_cap_mw'] = 3500

solution_duck = economic_dispatch_multi_time(gen_df_duck, loads_multi, gen_variable_multi)

In [ ]:
sol_gen_btm_duck = process_and_plot(solution_duck, gen_df, gen_variable_multi, T_period,
                                     'Stacked Generation with 3500 MW Solar (Duck Curve)')

In [ ]:
# Compare LMPs across all three scenarios
fig = go.Figure()
fig.add_trace(go.Scatter(x=solution_multi['lmps']['hour'], y=solution_multi['lmps']['lmp'],
                         name='Base (no ramps)', line=dict(dash='dot')))
fig.add_trace(go.Scatter(x=solution_ramp['lmps']['hour'], y=solution_ramp['lmps']['lmp'],
                         name='Base (with ramps)', line=dict(dash='dash')))
fig.add_trace(go.Scatter(x=solution_duck['lmps']['hour'], y=solution_duck['lmps']['lmp'],
                         name='3500 MW Solar (with ramps)', line=dict(width=3)))
fig.update_layout(title='LMP Comparison Across Scenarios',
                  xaxis_title='Hour', yaxis_title='LMP ($/MWh)',
                  width=1000, height=500)
fig.show()

Notice how with high solar penetration:
- **Midday LMPs collapse** as cheap solar displaces expensive gas generation
- **Evening LMPs spike** as solar drops off and gas plants must ramp up quickly
- The **price spread** between midday and evening increases dramatically

This price pattern is exactly what incentivizes **energy storage** investment in real markets.

### Ramp constraint marginal values

In [ ]:
marg_gen = solution_duck['marginals'].merge(gen_df[['r_id', 'resource']], on='r_id')
cc_marginals = marg_gen[marg_gen['resource'] == 'natural_gas_fired_combined_cycle']

fig = px.line(cc_marginals, x='hour', y='dual', color='r_id',
              title='Ramp Up Constraint Marginal Values (Combined Cycle)',
              labels={'dual': 'Marginal Value ($/MWh)'},
              width=800, height=500)
fig.show()

During the ramp hours, the marginal value becomes negative. The interpretation:

**If the RHS of the RampUp constraint were increased by 1 unit, the problem objective would reduce by that amount.**

This happens because ramp-limited combined cycle plants must be substituted by more expensive combustion turbines.

In [ ]:
# Show the most binding ramp constraint
worst_ramp = marg_gen.loc[marg_gen['dual'].idxmin()]
print(f"Most binding ramp constraint: r_id={worst_ramp['r_id']}, hour={worst_ramp['hour']}")
print(f"Marginal value: {worst_ramp['dual']:.4f} $/MWh")

# Verify by cost difference
varcost_comp = supply_curve[supply_curve['r_id'].isin([5, 9])][['r_id', 'varcost']]
print(f"\nCost difference between r_id=9 and r_id=5: {varcost_comp['varcost'].iloc[1] - varcost_comp['varcost'].iloc[0]:.4f} $/MWh")
print("(This should match the magnitude of the ramp marginal!)")

---
## Real-World Comparison: CAISO Data (2023-2024)

Let's now pull **actual CAISO generation and demand data** from 2023-2024 to see how real California operations compare with our model. We use the open-source [`gridstatus`](https://github.com/gridstatus/gridstatus) library to fetch this data directly.

*Note: The data download may take a minute. If `gridstatus` is unavailable, the cell will produce a message and you can skip to the next section.*

In [ ]:
try:
    import gridstatus
    caiso = gridstatus.CAISO()

    # Fetch a spring day in 2024 for comparison (April 11, 2024)
    caiso_demand = caiso.get_load(date="Apr 11, 2024", end="Apr 12, 2024")
    caiso_fuel_mix = caiso.get_fuel_mix(date="Apr 11, 2024", end="Apr 12, 2024")

    # Resample to hourly if data is sub-hourly
    caiso_demand = caiso_demand.set_index('Time')
    if len(caiso_demand) > 30:  # sub-hourly data
        caiso_demand_hourly = caiso_demand.resample('h').mean()
    else:
        caiso_demand_hourly = caiso_demand
    caiso_demand_hourly['hour_of_day'] = caiso_demand_hourly.index.hour

    caiso_fuel_mix = caiso_fuel_mix.set_index('Time')
    if len(caiso_fuel_mix) > 30:
        caiso_fuel_hourly = caiso_fuel_mix.resample('h').mean()
    else:
        caiso_fuel_hourly = caiso_fuel_mix
    caiso_fuel_hourly['hour_of_day'] = caiso_fuel_hourly.index.hour

    caiso_available = True
    print(f"Successfully loaded CAISO data!")
    print(f"Demand range: {caiso_demand_hourly['Load'].min():.0f} - {caiso_demand_hourly['Load'].max():.0f} MW")
    print(f"\nFuel mix columns: {[c for c in caiso_fuel_hourly.columns if c != 'hour_of_day']}")

except Exception as e:
    caiso_available = False
    print(f"Could not load CAISO data: {e}")
    print("Skipping real-world comparison. You can still run all model sections above.")

In [ ]:
if caiso_available:
    # Plot CAISO actual fuel mix as stacked area
    fuel_cols = [c for c in caiso_fuel_hourly.columns if c not in ['hour_of_day']]
    fuel_long = caiso_fuel_hourly[fuel_cols].reset_index().melt(
        id_vars='Time', var_name='fuel', value_name='MW'
    )

    fig = px.area(fuel_long, x='Time', y='MW', color='fuel',
                  title='Actual CAISO Generation Mix (April 11, 2024)',
                  width=1000, height=500)
    fig.show()
else:
    print("CAISO data not available. Skipping this plot.")

In [ ]:
if caiso_available:
    # Plot demand comparison
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=caiso_demand_hourly['hour_of_day'],
        y=caiso_demand_hourly['Load'],
        name='CAISO Actual (Apr 2024)', line=dict(width=3)
    ))
    fig.add_trace(go.Scatter(
        x=list(range(24)),
        y=loads_multi['demand'].values,
        name='SDG&E Model (spring day)', line=dict(width=3, dash='dash')
    ))
    fig.update_layout(
        title='Demand Comparison: CAISO Actual vs. Model',
        xaxis_title='Hour of Day',
        yaxis_title='Demand (MW)',
        width=1000, height=500
    )
    fig.show()
    print("Note: CAISO covers all of California (~30 GW peak), while our model covers only SDG&E (~3 GW peak).")
    print("The shape of the demand curve is what matters for comparison, not the absolute MW values.")
else:
    print("CAISO data not available. Skipping this plot.")

### Key observations from the real CAISO data

Compare the actual CAISO generation mix with your model results:

1. **Solar dominance at midday**: In the real CAISO data, solar provides a large fraction of midday generation, just as in our model.
2. **The duck curve is real**: The net load (demand minus solar and wind) creates the characteristic duck shape.
3. **Gas ramping**: Natural gas plants ramp up sharply in the evening, consistent with our ramp constraint analysis.
4. **Scale difference**: CAISO covers all of California (~30 GW peak) while our SDG&E model covers ~3 GW. The *patterns* are similar even though the scale differs.

---
## "Break the Model" Exercises

The best way to understand an optimization model is to see what happens when things go wrong. In this section, we deliberately create problematic scenarios to observe how the solver responds and build intuition about constraints.

---

### Exercise 1: Demand exceeds total capacity (Infeasibility)

**What happens if demand is higher than the total available generation capacity?**

---
### Quiz 6: Predict the outcome

The total installed capacity in our system is about 3,400 MW. If we set demand to 5,000 MW:
1. Will the solver find a solution?
2. If not, what does the solver tell us? What real-world situation does this represent?
3. How do real system operators handle this situation?

---

In [ ]:
# Total available capacity
total_cap = gen_df['existing_cap_mw'].sum()
print(f"Total installed capacity: {total_cap:.1f} MW")
print(f"Setting demand to 5,000 MW (exceeds capacity by {5000 - total_cap:.1f} MW)")
print("="*60)

# Create infeasible demand
loads_infeasible = pd.DataFrame({'demand': [5000]})
result_infeasible = economic_dispatch_single(gen_df, loads_infeasible, var_cf_single)

print(f"\nSolver status: {result_infeasible['status']}")
if result_infeasible['cost'] is None:
    print("\nThe model is INFEASIBLE. There is no way to meet 5,000 MW of demand")
    print("with only {:.0f} MW of capacity. In the real world, this would mean".format(total_cap))
    print("rolling blackouts or emergency imports from neighboring regions.")
    print("\nIn our OPF seminar, we will see how transmission interconnections")
    print("with neighboring regions help prevent this situation.")

### Exercise 2: Remove all ramp limits

**What if generators could ramp infinitely fast?**

This is equivalent to running the multi-period model *without* ramp constraints (which we already did!). Let's compare the costs and dispatch patterns directly.

In [ ]:
print("=" * 60)
print("COMPARISON: With vs. Without Ramp Constraints")
print("=" * 60)
print(f"\nCost WITHOUT ramp limits: ${solution_multi['cost']:>15,.2f}")
print(f"Cost WITH ramp limits:    ${solution_ramp['cost']:>15,.2f}")
print(f"Additional cost of ramps: ${solution_ramp['cost'] - solution_multi['cost']:>15,.2f}")
print(f"\nThis {(solution_ramp['cost'] - solution_multi['cost'])/solution_multi['cost']*100:.2f}% cost increase")
print("represents the 'price of inflexibility' -- the cost of engineering")
print("limitations on how fast generators can change output.")
print("\nIn practice, removing ramp limits is unrealistic.")
print("However, more flexible generators (like batteries) can reduce this cost.")

### Exercise 3: Set Pmin > 0 for thermal plants

In reality, thermal generators (gas, coal, nuclear) often have a **minimum stable output**. A gas turbine might need to run at a minimum of 30-50% of its capacity once it is turned on. Setting $Pmin = 0$ is an approximation we made for simplicity.

---
### Quiz 7: Predict the impact of Pmin > 0

If we set Pmin to 40% of Pmax for all gas generators:
1. Will the total cost increase or decrease? By how much (your guess)?
2. During midday when solar is abundant, what problem does a high Pmin create?
3. If gas plants *must* run at 40% minimum, and solar is producing more than enough... what gives?

*Hint: Think about what we assumed about Pmin for renewables.*

---

In [ ]:
def economic_dispatch_with_pmin(gen_df, loads, gen_variable, pmin_fraction=0.0):
    """
    ED with ramp constraints AND minimum generation constraints for thermal plants.
    pmin_fraction: fraction of Pmax that thermal plants must generate when on.
    
    NOTE: This is still a continuous relaxation (no binary on/off decision).
    The proper treatment requires unit commitment (integer variables),
    which we will cover in the next seminar.
    """
    model = pyo.ConcreteModel()
    model.dual = pyo.Suffix(direction=pyo.Suffix.IMPORT)

    G_var = list(gen_df[gen_df['is_variable'] == True]['r_id'])
    G_nonvar = list(gen_df[gen_df['is_variable'] == False]['r_id'])
    G = list(gen_df['r_id'])
    T = list(loads['hour'])
    T_red = T[:-1]

    model.G = pyo.Set(initialize=G)
    model.T = pyo.Set(initialize=T, ordered=True)

    gen_var_cf = gen_variable.merge(
        gen_df[gen_df['is_variable'] == True][['r_id', 'gen_full', 'existing_cap_mw']],
        on='gen_full', how='inner'
    )

    gen_lookup = gen_df.set_index('r_id')
    loads_lookup = loads.set_index('hour')

    # Decision variables
    model.GEN = pyo.Var(model.G, model.T, within=pyo.NonNegativeReals)

    # Objective
    def obj_rule(m):
        return (
            sum(
                (gen_lookup.loc[i, 'heat_rate_mmbtu_per_mwh'] * gen_lookup.loc[i, 'fuel_cost'] +
                 gen_lookup.loc[i, 'var_om_cost_per_mwh']) * m.GEN[i, t]
                for i in G_nonvar for t in T
            ) +
            sum(
                gen_lookup.loc[i, 'var_om_cost_per_mwh'] * m.GEN[i, t]
                for i in G_var for t in T
            )
        )
    model.objective = pyo.Objective(rule=obj_rule, sense=pyo.minimize)

    # Demand constraint
    model.cDemand = pyo.Constraint(model.T)
    for t in T:
        model.cDemand[t] = sum(model.GEN[i, t] for i in G) == loads_lookup.loc[t, 'demand']

    # Capacity constraints (non-variable) -- upper bound
    model.cap_nonvar_upper = pyo.ConstraintList()
    for i in G_nonvar:
        for t in T:
            model.cap_nonvar_upper.add(model.GEN[i, t] <= gen_lookup.loc[i, 'existing_cap_mw'])

    # Minimum generation constraints for thermal plants
    model.pmin = pyo.ConstraintList()
    for i in G_nonvar:
        for t in T:
            model.pmin.add(
                model.GEN[i, t] >= pmin_fraction * gen_lookup.loc[i, 'existing_cap_mw']
            )

    # Variable generation capacity constraints
    model.cap_var = pyo.ConstraintList()
    for _, row in gen_var_cf.iterrows():
        model.cap_var.add(
            model.GEN[row['r_id'], row['hour']] <= row['cf'] * row['existing_cap_mw']
        )

    # Ramp constraints
    model.ramp_up = pyo.Constraint(model.G, T_red)
    model.ramp_dn = pyo.Constraint(model.G, T_red)
    for i in G:
        for idx, t in enumerate(T_red):
            t_next = T[idx + 1]
            model.ramp_up[i, t] = (
                model.GEN[i, t_next] - model.GEN[i, t] <=
                gen_lookup.loc[i, 'existing_cap_mw'] * gen_lookup.loc[i, 'ramp_up_percentage']
            )
            model.ramp_dn[i, t] = (
                model.GEN[i, t] - model.GEN[i, t_next] <=
                gen_lookup.loc[i, 'existing_cap_mw'] * gen_lookup.loc[i, 'ramp_dn_percentage']
            )

    # Solve
    solver = pyo.SolverFactory('appsi_highs')
    result = solver.solve(model, tee=False)

    status = str(result.solver.termination_condition)
    if status != 'optimal':
        print(f"\n*** Solver status: {status} ***")
        return {'solution': pd.DataFrame(), 'cost': None, 'lmps': pd.DataFrame(), 'status': status}

    records = []
    for i in G:
        for t in T:
            records.append({'r_id': i, 'hour': t, 'gen': pyo.value(model.GEN[i, t])})

    lmps = pd.DataFrame({'hour': T, 'lmp': [model.dual[model.cDemand[t]] for t in T]})

    return {
        'solution': pd.DataFrame(records),
        'cost': pyo.value(model.objective),
        'lmps': lmps,
        'status': status,
    }

In [ ]:
# Test with Pmin = 40% for thermal plants (base solar)
print("Solving with Pmin = 40% of Pmax for all thermal generators...")
solution_pmin = economic_dispatch_with_pmin(gen_df, loads_multi, gen_variable_multi, pmin_fraction=0.4)
print(f"Status: {solution_pmin['status']}")
if solution_pmin['cost']:
    print(f"Cost: ${solution_pmin['cost']:,.2f}")

In [ ]:
# Now try with high solar AND Pmin -- this can become infeasible!
print("Solving with Pmin = 40% AND 3500 MW solar...")
print("(This may be infeasible if thermal minimum output + solar > demand at midday)")
print("=" * 60)

solution_pmin_duck = economic_dispatch_with_pmin(gen_df_duck, loads_multi, gen_variable_multi, pmin_fraction=0.4)
print(f"\nStatus: {solution_pmin_duck['status']}")

if solution_pmin_duck['status'] != 'optimal':
    print("\nThe model is INFEASIBLE! This is because:")
    min_thermal = gen_df[gen_df['is_variable'] == False]['existing_cap_mw'].sum() * 0.4
    print(f"  - Minimum thermal output (40%): {min_thermal:.0f} MW")
    print(f"  - Minimum demand: {loads_multi['demand'].min():.0f} MW")
    print(f"  - Plus solar can add up to ~2,000 MW at midday")
    print(f"  - Total minimum supply can exceed demand!")
    print("\nIn reality, this is the 'over-generation' problem that CAISO faces.")
    print("Solutions include: curtailing renewables, exporting power, or shutting")
    print("down thermal units -- which requires unit commitment (next seminar!).")
else:
    print(f"Cost: ${solution_pmin_duck['cost']:,.2f}")
    sol_gen_btm_pmin = process_and_plot(solution_pmin_duck, gen_df, gen_variable_multi, T_period,
                                        'Generation with Pmin=40% and 3500 MW Solar')

**Key insight**: The Pmin constraint is a simplified version of the **unit commitment** problem. In reality, generators are either ON (operating between Pmin and Pmax) or OFF (generating 0). This ON/OFF decision requires **binary variables**, making it a mixed-integer program (MIP). We will cover this in the **Unit Commitment seminar**.

---
## Scenario Comparison Table

Let's now compile a summary table comparing all the scenarios we have run, side by side.

---

In [ ]:
# Build comparison table
scenarios = {}

# 1. Base case (no ramps)
scenarios['Base (no ramps)'] = solution_multi

# 2. Base case (with ramps)
scenarios['Base (with ramps)'] = solution_ramp

# 3. Duck curve (3500 MW solar, with ramps)
scenarios['3500 MW Solar'] = solution_duck

# 4. Pmin = 40% (base solar)
if solution_pmin['cost'] is not None:
    scenarios['Pmin=40% (base solar)'] = solution_pmin

# Build table rows
rows = []
for name, sol in scenarios.items():
    lmps = sol['lmps']
    row = {
        'Scenario': name,
        'Total Cost ($)': f"${sol['cost']:,.0f}",
        'Avg LMP ($/MWh)': f"${lmps['lmp'].mean():.2f}",
        'Min LMP ($/MWh)': f"${lmps['lmp'].min():.2f}",
        'Max LMP ($/MWh)': f"${lmps['lmp'].max():.2f}",
        'LMP Spread': f"${lmps['lmp'].max() - lmps['lmp'].min():.2f}",
        'Status': sol['status'],
    }
    rows.append(row)

# Add infeasible scenarios
if result_infeasible['cost'] is None:
    rows.append({
        'Scenario': 'Demand > Capacity',
        'Total Cost ($)': 'N/A',
        'Avg LMP ($/MWh)': 'N/A',
        'Min LMP ($/MWh)': 'N/A',
        'Max LMP ($/MWh)': 'N/A',
        'LMP Spread': 'N/A',
        'Status': result_infeasible['status'],
    })

if solution_pmin_duck['cost'] is None:
    rows.append({
        'Scenario': 'Pmin=40% + 3500MW Solar',
        'Total Cost ($)': 'N/A',
        'Avg LMP ($/MWh)': 'N/A',
        'Min LMP ($/MWh)': 'N/A',
        'Max LMP ($/MWh)': 'N/A',
        'LMP Spread': 'N/A',
        'Status': solution_pmin_duck['status'],
    })

comparison_df = pd.DataFrame(rows)
comparison_df

### Reading the comparison table

Key takeaways:

1. **Ramp constraints increase cost**: The "price of inflexibility" -- more expensive generators must substitute for cheaper ones that can't ramp fast enough.

2. **More solar reduces average cost** but increases the **LMP spread** (difference between peak and off-peak prices). This price signal incentivizes storage and demand response.

3. **Pmin constraints can cause infeasibility** when combined with high renewable penetration. This is the **over-generation problem** that motivates:
   - Renewable curtailment
   - Energy storage
   - Unit commitment (turning plants OFF entirely)
   - Demand response

4. **Excess demand causes infeasibility** -- the system simply cannot meet the load. Real-world solutions include:
   - Emergency imports via transmission lines (covered in OPF seminar)
   - Demand-side management / load shedding
   - Peaker plants and demand response (covered in UC seminar)

---
### Quiz 8: Final Reflection

1. In our model, the LMP is determined by the dual of the demand constraint. In a real market with transmission constraints (OPF), why would LMPs differ across different locations (nodes) in the grid?

2. We assumed all generators have Pmin = 0, meaning they can smoothly reduce output to zero. Why is this unrealistic for a coal or nuclear plant? What kind of optimization problem do we need to handle the ON/OFF decision?

3. Looking at the comparison table, the "LMP Spread" increases dramatically with high solar. If you were an energy storage investor, how would you use this information?

4. Our model found two scenarios that were **infeasible**. In real power systems, infeasibility is not an option -- the lights must stay on. What "safety valves" do real system operators use? *(We will explore these in the UC and OPF seminars.)*

---

## Summary

In this seminar, we have:

- Built an **economic dispatch model** from scratch using Pyomo and HiGHS
- Progressed from single-period to multi-period models with increasing realism
- Extracted **Locational Marginal Prices (LMPs)** from constraint duals
- Explored the **duck curve** and its implications for system operations
- Compared model results with **real CAISO data** from 2023-2024
- Identified **model limitations** by deliberately breaking it

### What's next?

- **Unit Commitment**: Adding binary ON/OFF decisions for generators, startup costs, minimum up/down times, and proper Pmin handling
- **Optimal Power Flow**: Adding network topology, transmission constraints, and spatially varying LMPs